# Initialize

In [1]:
# import packages 

%load_ext autoreload

import os
from os import listdir
from os.path import isfile, join

import gc
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import pickle
import logging

# Imports for Deep Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error, confusion_matrix, 
    ConfusionMatrixDisplay, classification_report, 
    cohen_kappa_score, matthews_corrcoef)
from sklearn.utils.class_weight import compute_class_weight

import keras
from keras import metrics
from keras.regularizers import l1
from keras.models import Sequential, load_model
from keras import initializers
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import (Dense, Dropout, Activation, Flatten, Input, 
                          TimeDistributed, Reshape, Permute, Flatten, 
                          RepeatVector, Bidirectional, InputLayer,  
                          AlphaDropout, Normalization, MaxPooling2D, Embedding, 
                          ConvLSTM1D, Attention, TimeDistributed, LocallyConnected1D,
                          LSTM, GRU)
from keras.models import Model, load_model
from keras import optimizers
from keras.utils.vis_utils import plot_model
from tensorflow.keras.optimizers import Nadam, Adam, SGD, Adadelta, Adamax, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import tensorflow as tf

from collections import deque
import random
import math
import time 
import datetime

from keras.utils import np_utils
import shutil

from sklearn.preprocessing import normalize, StandardScaler
from sklearn.decomposition import PCA

import matplotlib

# Imports for CSV Processing
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches

print(keras.__version__)
print(matplotlib.__version__)

%load_ext tensorboard

2.12.0
3.7.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ensure GPU RAM is >10 GB

!nvidia-smi

print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

GPU name:  []


In [ ]:
# Consistent random seed selection improves reliability of Keras training performance
tf.random.set_seed(42)
np.random.seed(42)

Run the below cell if using git repo cloned to Google Drive

In [ ]:
%cd ''
basepath='./drive/MyDrive/Classifier Final/'
%cd '/content/drive/MyDrive/Classifier Final/'
path_Fourier_scored='./Variables/Feats_Fourier_and_PSD/'
path_Fourier_unscored='./Variables/Feats_Fourier_and_PSD_unscored/'
path_output='./CSV_Outputs/'
path_variables='./Variables/'

/root
/content/drive/MyDrive/Classifier Final


Local variant of above cell

In [ ]:
# basepath='C:/Users/BHARVE4/Documents/deep-sleep-seizure/Classifier_Final'
# %cd $basepath
# path_Fourier_scored='./Variables/Feats_Fourier_and_PSD/'
# path_Fourier_unscored='./Variables/Feats_Fourier_and_PSD_unscored/'
# path_output='./CSV_Outputs/'
# path_variables='./Variables/'

C:\Users\BHARVE4\Documents\deep-sleep-seizure\Classifier_Final


In [ ]:
# Unzip Fourier Transformed variables to folders if needed 
# !unzip '/content/drive/MyDrive/Classifier Final/Variables/Feats_Fourier_and_PSD.zip' -d '/content/drive/MyDrive/Classifier Final/Variables/'
# !unzip '/content/drive/MyDrive/Classifier Final/Variables/Feats_Fourier_and_PSD_unscored.zip' -d '/content/drive/MyDrive/Classifier Final/Variables/'
# !unzip '/content/drive/MyDrive/Classifier Final/Pretrained/reports.zip' -d '/content/drive/MyDrive/Classifier Final/Pretrained/'

Archive:  /content/drive/MyDrive/Classifier Final/Variables/Feats_Fourier_and_PSD.zip
Archive:  /content/drive/MyDrive/Classifier Final/Variables/Feats_Fourier_and_PSD_unscored.zip


KeyboardInterrupt: ignored

# Define Functions

In [ ]:
def consecutive(data, stepsize=1):
    inds =  np.where(np.diff(data) != stepsize)[0]+1
    consec = np.split(data, inds)
    return consec, inds

In [ ]:
def unique(list1): 
  
    # intilize a null list 
    global unique_list  
    unique_list = [] 
    
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    return sorted(unique_list)

In [ ]:
def load_Fourier_xy(path, file_list):

    total_files = len(file_list)  # length of list
    print(total_files)
    
    epochs = 2160    
    total_epoch_count = int(epochs*(total_files)) # total length of data array in 10 second epochs
    
    # establish x and y vectors for input data
    dh1 = display(f'Items left: {total_files}',display_id=True)
    dh2 = display('Loading...',display_id=True)

    # counting and progress flag variables 
    y_done = 0
    x_done = 0
    file_counter = 0
    
    x = np.zeros((total_epoch_count, 100))
    y = np.zeros((total_epoch_count, 1))
    excl = 0
    # iterate over all Fourier transformed files
    for item in (sorted(file_list)):
        # print(item)
        total_files -= 1
        dh1.update(f'Items left: {total_files}')
        
        item = item.replace("x_ffnorm", "y_ffnorm") 
        current_file_y = np.load(f"{path}{item}")
        item = item.replace("y_ffnorm", "x_ffnorm") 

        for i in current_file_y:
          if i not in [1,2,3,4,5]:
            excl=1

        if excl==0:
            # load Fourier feature vectors
            current_file_x = np.load(f"{path}{item}")
            x[file_counter*epochs:((file_counter+1)*epochs)] = StandardScaler().fit_transform(current_file_x)
            x_done = 1
            
            # load Sleep/Seizure score vectors
            item = item.replace("x_ffnorm", "y_ffnorm") 
            current_file_y = np.load(f"{path}{item}")
            y[file_counter*epochs:((file_counter+1)*epochs)] = current_file_y-1
            y_done = 1

            if x_done == 1 and y_done == 1:
                file_counter += 1
                dh2.update(f'loaded x and y for {item}')
                if file_counter%50 == 0:
                    print(file_counter)
                x_done = 0
                y_done = 0

        else: 
            print(f'{item} excluded')
            excl = 0

    return x,y

In [ ]:
def norm_sklearn_classweight(y_train, mu=False):
    
    # Get maximum range of class labels 
    classes = unique(np.argmax(y_train,1))

    # Convert scores back from one-hot to ints
    y_train_classes = [np.argmax(z) for z in y_train]
    unique(y_train_classes)

    
    # assign balanced class weights
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_classes)
    print(weights)
    class_weight = {}
    ind = 0

    
    # adjust weights based upon mu parameter or the minimum weight present
    for i in classes:
        j = int(np.where(classes==i)[0])
        print(j)
        if mu == False:
            score = weights[j]/min(weights)  
        else: 
            score = math.log(weights[j]/mu)
        class_weight[j] = score if score > 1.0 else 1.0

    return class_weight

In [ ]:
def create_train_BiLSTM_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, n_iter=200, batch_factor=.0001, prefix="", layers=4, basepath=''):
        
        METRICS = ['accuracy',
          metrics.TruePositives(name='tp'),
          metrics.FalsePositives(name='fp'),
          metrics.TrueNegatives(name='tn'),
          metrics.FalseNegatives(name='fn'),
          metrics.CategoricalAccuracy(name='categorical_accuracy'),
          metrics.Precision(name='precision'),
          metrics.Recall(name='recall'),
          metrics.AUC(name='auc', curve="PR"),
          metrics.CategoricalCrossentropy(name='categorical_crossentropy')]
      
        data_width = X_train.shape[1]
        data_depth = X_train.shape[2]
        shape = (data_width,data_depth)

        batch_size = 2160
        LSTM_size = n_iter
        rs_flag=True
            
        # Model creation with optional stacking of Bi-LSTM/Dropout pairs
        model = Sequential()
        model.add(InputLayer(input_shape=shape))

        for count in range(layers):
            if count==0:
              activity_regularizer=l1(0.0001)
            else:
              activity_regularizer=None
          
            factor = 2**(count)
            forward_layer = LSTM(int(LSTM_size/factor), activity_regularizer=activity_regularizer, return_sequences=rs_flag)
            backward_layer = LSTM(int(LSTM_size/factor), activity_regularizer=activity_regularizer, return_sequences=rs_flag,
                                  go_backwards=True)
            model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
            model.add(Dropout(.4))

        # Dense softmax prediction
        model.add(Flatten())     
        model.add(Dense(5))
        model.add(Activation('softmax'))


        # model parameters
        opt=Nadam(learning_rate=0.00001)
        
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=METRICS)
        model.build()
        model.summary()
        
        # model logging parameters
        model_name=f"BiLSTM_size_{LSTM_size}_{prefix}_"
        filepath = f"{model_name}"+"_weights.auc:{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')       
                              
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)
                    
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # call model.fit
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name

In [ ]:
def create_train_LSTM_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, n_iter=200, batch_factor=.0001, prefix="", layers=4, basepath=''):
        
        METRICS = ['accuracy',
          metrics.TruePositives(name='tp'),
          metrics.FalsePositives(name='fp'),
          metrics.TrueNegatives(name='tn'),
          metrics.FalseNegatives(name='fn'),
          metrics.CategoricalAccuracy(name='categorical_accuracy'),
          metrics.Precision(name='precision'),
          metrics.Recall(name='recall'),
          metrics.AUC(name='auc', curve="PR"),
          metrics.CategoricalCrossentropy(name='categorical_crossentropy')]
      
        data_width = X_train.shape[1]
        data_depth = X_train.shape[2]
        shape = (data_width,data_depth)

        batch_size = 2160
        LSTM_size = n_iter
        rs_flag=True
            
        # Model creation with optional stacking of Bi-LSTM/Dropout pairs
        model = Sequential()
        model.add(InputLayer(input_shape=shape))

        for count in range(layers):
            if count==0:
              activity_regularizer=l1(0.0001)
            else:
              activity_regularizer=None
            factor = 2**(count)
            forward_layer = LSTM(int(LSTM_size/factor), activity_regularizer=activity_regularizer, return_sequences=rs_flag)
            backward_layer = LSTM(int(LSTM_size/factor), activity_regularizer=activity_regularizer, return_sequences=rs_flag,
                                  go_backwards=True)
            model.add(forward_layer)
            model.add(Dropout(.4))

        # Dense softmax prediction
        model.add(Flatten())     
        model.add(Dense(5))
        model.add(Activation('softmax'))


        # model parameters
        opt=Nadam(learning_rate=0.00001)
        
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=METRICS)
        model.build()
        model.summary()
        
        # model logging parameters
        model_name=f"Single_LSTM_size_{LSTM_size}_{prefix}_"
        filepath = f"{model_name}"+"_weights.auc:{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')       
                              
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)
                    
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # call model.fit
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name

In [ ]:
def create_train_flat_BiLSTM_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, n_iter=8, batch_factor=.0001,prefix="", layers=3,basepath=''):

        METRICS = ['accuracy',
          metrics.TruePositives(name='tp'),
          metrics.FalsePositives(name='fp'),
          metrics.TrueNegatives(name='tn'),
          metrics.FalseNegatives(name='fn'),
          metrics.CategoricalAccuracy(name='categorical_accuracy'),
          metrics.Precision(name='precision'),
          metrics.Recall(name='recall'),
          metrics.AUC(name='auc', curve="PR"),
          metrics.CategoricalCrossentropy(name='categorical_crossentropy')]         

        data_width=X_train.shape[1]
   
        if len(X_train.shape)>2:
            data_depth=X_train.shape[2]
            shape=(data_width,data_depth)
        else: 
            data_width=X_train.shape[1]
            shape=(data_width,)
            
        batch_size=2160
        LSTM_size=n_iter
        rs_flag=False
        
        # Model creation with optional stacking of Bi-LSTM/Dropout pairs
        model = Sequential()
        model.add(InputLayer(input_shape=shape))
        model.add(Flatten())           
        model.add(RepeatVector(1)) 

        # Each Bi-LSTM in cascade gets smaller by a factor of 2
        if layers in range(1,5):
            forward_layer = LSTM(int(LSTM_size), return_sequences=rs_flag)
            backward_layer = LSTM(int(LSTM_size), return_sequences=rs_flag,
                                  go_backwards=True)
            model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
            model.add(RepeatVector(1)) 
            model.add(Dropout(.4))

            factor = 2
            if layers in range (2,5):
              forward_layer = LSTM(int(LSTM_size/factor), return_sequences=rs_flag)
              backward_layer = LSTM(int(LSTM_size/factor), return_sequences=rs_flag,
                                    go_backwards=True)
              model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
              model.add(Dropout(.4))
              model.add(RepeatVector(1)) 

              factor = 4
              if layers in range (3,5):
                forward_layer = LSTM(int(LSTM_size/factor), return_sequences=rs_flag)
                backward_layer = LSTM(int(LSTM_size/factor), return_sequences=rs_flag,
                                      go_backwards=True)
                model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
                model.add(Dropout(.4))
                model.add(RepeatVector(1)) 

                factor = 8
                if layers in range (4,5):
                    forward_layer = LSTM(int(LSTM_size/factor), return_sequences=rs_flag)
                    backward_layer = LSTM(int(LSTM_size/factor), return_sequences=rs_flag,
                                          go_backwards=True)
                    model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
                    model.add(Dropout(.4))
                    model.add(RepeatVector(1)) 
                
        # Dense softmax prediction
        model.add(Flatten())     
        model.add(Dense(5))
        model.add(Activation('softmax'))


        # model parameters
        opt=Nadam(learning_rate=0.00001)
        
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=METRICS)
        model.build()
        model.summary()
        
        model_name=f"flat_LSTM_size_{LSTM_size}_{prefix}_"
        filepath = f"{model_name}"+"_weights.auc:{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        # model logging parameters
        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')       
                              
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)
                    
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        
        csv_path=f"{model_name}_flat_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # call model.fit
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])


        return model, history, logpath, csv_path, model_name

In [ ]:
def create_train_CNN_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, n_iter=8, batch_factor=.0001,prefix="", layers=3, basepath=''):
        
        METRICS = ['accuracy',
          metrics.TruePositives(name='tp'),
          metrics.FalsePositives(name='fp'),
          metrics.TrueNegatives(name='tn'),
          metrics.FalseNegatives(name='fn'),
          metrics.CategoricalAccuracy(name='categorical_accuracy'),
          metrics.Precision(name='precision'),
          metrics.Recall(name='recall'),
          metrics.AUC(name='auc', curve="PR"),
          metrics.CategoricalCrossentropy(name='categorical_crossentropy')]

        data_width=X_train.shape[1]
   
        if len(X_train.shape)>2:
            data_depth=X_train.shape[2]
            shape=(data_width,data_depth)
        else: 
            shape=(data_width,)
            
        # parameters
        batch_size=2160
        CNN_size=n_iter
        rs=True
            
        # create CNN using Keras.Sequential
        model = Sequential()
        model.add(InputLayer(input_shape=shape))
        model.add(Conv1D(int(CNN_size),2))
        model.add(Flatten())           
        model.add(Dense(5))
        model.add(Activation('softmax'))

        opt=Nadam(learning_rate=0.00001)
        
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=METRICS)
        model.build()
        model.summary()
        
        # model logging parameters
        model_name=f"CNN_size_{CNN_size}_{prefix}_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        filepath = f"{model_name}"+"_weights.auc:{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"
        
        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')       
                              
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)
                    
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # call model.fit
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name

In [ ]:
def create_train_Dense_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, n_iter=8, batch_factor=.0001,prefix="", layers=3, basepath=''):
        

        name_part='Both'
        data_width=X_train.shape[1]
   
        if len(X_train.shape)>2:
            data_depth=X_train.shape[2]
            shape=(data_width,data_depth)
        else: 
            data_width=X_train.shape[1]
            shape=(data_width,)
            # shape=(20,data_width)
        Dense2_char=[]
            
        
        METRICS = [
        'accuracy',
        metrics.TruePositives(name='tp'),
        metrics.FalsePositives(name='fp'),
        metrics.TrueNegatives(name='tn'),
        metrics.FalseNegatives(name='fn'),
        metrics.CategoricalAccuracy(name='categorical_accuracy'),
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall'),
        metrics.AUC(name='auc', curve="PR"),
        metrics.CategoricalCrossentropy(name='categorical_crossentropy')]
        
        # batch_size=int((X_train.shape[0])*batch_factor)
        batch_size=2160
        layer_size_conv=64         
        LSTM_size=n_iter
        rs=True
            
        
        model = Sequential()

        model.add(InputLayer(input_shape=shape))
        model.add(Dense(n_iter))

        model.add(Flatten())           

        model.add(Dense(5))
        model.add(Activation('softmax'))


        opt=Nadam(learning_rate=0.00001)
        
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=METRICS)
        
        model.build()

        model.summary()
        
        model_name=f"Dense_size_{LSTM_size}_{prefix}_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        filepath = f"{model_name}"+"_weights.auc:{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')


                        
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.0001,restore_best_weights=True)

        
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name


In [ ]:
def save_model_report_seq(model, model_history, ctrls=0, run=2, x=[], y=[], X_train_seq=[], X_held_seq=[], y_train_seq=[], y_held_seq=[], X_test_seq=[], y_test_seq=[], model_name='test',i=1,model_num=8,basepath='',logpath='', csvpath='',savepath='./Results/'):  
  
  print(model_name)
  model_num=i*8

  if ctrls==1:
    num_classes=3
  else:
    num_classes=5

  if ctrls==0:
    charpath=f'Kainic Acid'
  elif ctrls==1:
    charpath=f'Control'
  elif ctrls==2:
    charpath='AllData'
  
  if os.path.exists(savepath)==False:
    os.mkdir(savepath)
  if os.path.exists(f'{savepath}Models/')==False:
    os.mkdir(f'{savepath}Models/')
  if os.path.exists(f'{savepath}Softmax/')==False:
    os.mkdir(f'{savepath}Softmax/')

  codes=np.array(['wake','nrem','rem','seizure','post-ictal'])

  # save model file
  # model.save(f'{savepath}Model_{model_name}.h5')  

  if os.path.exists(f'{savepath}Models/{model_name}.h5')==False:
   model.save(f'{savepath}Models/{model_name}.h5')  

  if model_history!=None:
    # save model history
    with open(f'{savepath}{model_name}{model_num}historydict','wb') as file_pi:
        pickle.dump(model_history.history, file_pi)
      
    shutil.copyfile(logpath+csvpath, f'{savepath}{csvpath}')
    
  # reconstitute entire dataset for evaluation
  x_new=np.concatenate((X_held_seq,X_test_seq, X_train_seq))
  y_new_1hot=np.concatenate((y_held_seq,y_test_seq, y_train_seq))
  
  # with model report file open 
  with open(f'{savepath}Report_{model_name}.txt','w') as f:
    # iterate over train/test/val and combined datasets 
    for selection in range(0,4):
        gc.collect()
        names=['Saline-KA Validation Dataset', 'Saline Control Dataset', 'Training Dataset', 'Train-Test-Val']
        X_check, y_check = [[X_held_seq, y_held_seq], [X_test_seq, y_test_seq],[X_train_seq, y_train_seq], [x_new, y_new_1hot]][selection]
  
        y_pred=model.predict(X_check, batch_size=2160)
        # y_pred.tofile(f'{savepath}Softmax/results_{model_name}.csv', sep=',')

        #plot confusion matrices
        y_pred_codes=np.argmax(y_pred,1)
        y_test_codes=np.argmax(y_check,1)
        print(f'\n {names[selection]} \n', file=f)
        print(classification_report(y_test_codes, y_pred_codes), file=f)

        dict1 = classification_report(y_test_codes, y_pred_codes, output_dict=True)
        df = pd.DataFrame(data=dict1)
        df = (df.T)
        print (df)

        dataset_name=names[selection]

        df.to_excel(f'{savepath}{dataset_name}{model_name}.xlsx')
        
        cm=confusion_matrix(y_test_codes,y_pred_codes,normalize='true')

        print(f'unique ground truth labels: {unique(y_test_codes)}')
        print(f'unique predicted labels: {unique(y_pred_codes)}')


        if len(unique(y_pred_codes))>len(unique(y_test_codes)):
          disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=codes[unique(y_pred_codes)])
        else:
          disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=codes[unique(y_test_codes)])

        disp.plot()

        print(f'{names[selection]}')

        plt.title(f'Confusion Matrix for {names[selection]}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'{savepath}{model_name}_{names[selection]}_CM_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.png', dpi=700)

        plt.close()
        gc.collect()


In [ ]:
def class_matrix(model, ctrls=0, run=2, x=[], y=[], model_name='test',model_num=8, LSTM_size=None, basepath=''):
  run=f'Test_{run}_'
  if ctrls==1:
    num_classes=3
  else:
    num_classes=5

  codes=[None]*5
  codes[0]='Wake'
  codes[1]='NREM'
  codes[2]='REM'
  codes[3]='Seizure'
  codes[4]='Post-Ictal'

  labels_dict={}

  if ctrls==0:
    charpath=f'Kainic Acid'
  elif ctrls==1:
    charpath=f'Control'
  elif ctrls==2:
    charpath='AllData'

  savepath=f'./Trained/Final/'

  model_num=100*8

  Reportnum=1
  model_name=f"LSTM_size_{LSTM_size}_{model_name}_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  model.save(f'{savepath}_Model_{model_name}{model_num}.h5')  

  #savepath=f'{savepath}{model.name}/{charpath}/'

  if os.path.exists(savepath)==False:
   os.mkdir(savepath)

  Reportnum=1
  # with open(f'{savepath}{model_name}{model_num}historydict','wb') as file_pi:
  #     pickle.dump(model_history.history, file_pi)

  x_new=np.concatenate((X_held_seq,X_test_seq, X_train_seq))
  y_new_1hot=np.concatenate((y_held_seq,y_test_seq, y_train_seq))


  tests=4
  with open(f'{savepath}Report_{model_name}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.txt','w') as f:
    for sel in range(0,4):
        names=['Saline-KA Validation Dataset', 'Saline Control Dataset', 'Training Dataset', 'Train-Test-Val']
        X_check, y_check = [[X_held_seq, y_held_seq], [X_test_seq, y_test_seq],[X_train_seq, y_train_seq], [x_new, y_new_1hot]][sel]

        y_pred=model.predict(X_check, batch_size=int(2160/2))
  
        y_pred_codes=np.argmax(y_pred,1)
        y_test_codes=np.argmax(y_check,1)
        print(f'\n {names[sel]} \n')
        f.write(classification_report(y_test_codes, y_pred_codes))

        cm=confusion_matrix(y_test_codes,y_pred_codes, normalize='true')

        disp=ConfusionMatrixDisplay(confusion_matrix=cm)

        disp.plot()
        # plt.show()

        print(f'{names[sel]}')
        #cax = ax.matshow(cm)
        plt.title(f'Confusion Matrix for {names[sel]}')

        maxlabel=max(y_pred_codes)

        if sel==1:
          maxlabel=3

        disp.ax_.set_xticklabels(codes[0:maxlabel+1])
        disp.ax_.set_yticklabels(codes[0:maxlabel+1])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'{savepath}{model_name}_{names[sel]}_CM_.png', dpi=700)

        plt.show()
        plt.close()

In [ ]:
def generate_sequences(input_array, windows, x_or_y, max_feats=None, make_seq=True):

  if make_seq==True:

    if x_or_y not in ['X', 'Y']:
      "Please designate whether input is X or Y array using x_or_y parameter"
      return 

    for window in [windows]: # for loop to test varying window lengths


      classes=5

      shift=window*2 # All x variable rows will be sampled with sliding sequences 
      # If analyzing epoch 3, epochs from from -window_length (0) to 
      # +window_length (6) around each epoch of interest will be sampled
      # Therefore, the windowing cannot begin earlier than epoch # [window_length]

      # Windowing 

      if x_or_y in ['X']:
        if max_feats == None:
          max_feats=input_array.shape[1]
              # maximum number of features to use - use .shape[1] 
              # of X_train, or the number 100, for full-featured training
        output_array=np.zeros((len(input_array)-shift,window*2+1,max_feats))
        for i in range(window,len(input_array)-window):
          output_array[i-window]=input_array[i-window:i+window+1,0:max_feats]


      if x_or_y in ['Y']:
        output_array=np.zeros((len(input_array)-shift, classes))
        for i in range(window,len(input_array)-window):
          output_array[i-window]=input_array[i]


      gc.collect()

      return output_array

  elif make_seq==False:
    print('no sequence generated')
    return input_array

In [ ]:
# Rechtshaffen and Kales scoring layer

def R_and_K_evaluation(X_array_seq, model):
  if len(X_array_seq.shape)<2:
    X_array_seq=generate_sequences(X_array_seq, 3, x_or_y='X', max_feats=X_array_seq.shape()[1])

  y_pred=model.predict(X_array_seq, batch_size=2160)
  y_pred_scores = np.argmax(y_pred,1)

  y_pred_scores_baseline = y_pred_scores

  for idx in range(0,len(y_pred_scores)):
    if idx>0:
      if y_pred_scores[idx]==2 and y_pred_scores[idx-1]==0:
        y_pred_scores[idx]=0

      if y_pred_scores[idx]==1:
        if y_pred_scores[idx-1]!=1 and y_pred_scores[idx+1]!=1:
          y_pred_scores[idx]=y_pred_scores[idx-1]

      if y_pred_scores[idx]==0:
        if y_pred_scores[idx-1]!=0 and y_pred_scores[idx+1]!=0:
          y_pred_scores[idx]=y_pred_scores[idx-1]
      
  agreement = y_pred_scores == y_pred_scores_baseline

  agree_pct=len(agreement[agreement==True])/len(agreement)
  print(len(agreement[agreement==False]))
  print(agree_pct)

In [ ]:
def print_conf_mat(y_test_codes, y_pred_codes, legend, output_name):

      
        labelmax=max(max(y_test_codes),max(y_pred_codes))
        print(labelmax)
        print(codes[0:labelmax+1])

        disp=ConfusionMatrixDisplay.from_predictions(y_test_codes,y_pred_codes, labels=[0,1,2,3,4], normalize='true', display_labels=codes)

        plt.title(f'{legend}')
        plt.xlabel('Predicted')
        plt.ylabel('True')

        plt.show()
        plt.savefig(f'{output_name}_ConfMat.png', dpi=700)


In [ ]:
def score_data(path, errors, load_dropped_data=0, held_dates=[], scored=0):
    model.summary()

    path_Fourier_scored='./Variables/Feats_Fourier_and_PSD/'
    path_Fourier_unscored='./Variables/Feats_Fourier_and_PSD_unscored/' 

    print(scored)
    if scored==1:
      strpart='trainset_classifier'
      path_Fourier=path_Fourier_scored
    elif scored==2:
      strpart='trainset_expert'
      path_Fourier=path_Fourier_scored
    else:
      strpart='classifier_prev_un'
      path_Fourier=path_Fourier_unscored


    %matplotlib inline
    score=0
    good=0
    path_csv=f'./CSV_Outputs/csv_{strpart}_scored/'
    if os.path.isdir(path_csv) == False:
      os.mkdir(path_csv)


    print(path_Fourier)
    fourier_files = [f for f in listdir(path_Fourier) if isfile(join(path_Fourier, f))]

    fourier_files = [f for f in fourier_files if "x_ffnorm" in f]   # find all x arrays in fourier_files

    print(held_dates)

    if load_dropped_data==1:
        print('loading dropped data')
        fourier_files_2=[]
        for d in held_dates:
            fourier_files=[f for f in fourier_files if d in f]
        fourier_files_2=fourier_files
    else:
      for d in held_dates:
        fourier_files = [f for f in fourier_files if d not in f]
      fourier_files_2=fourier_files

    test_length=len(fourier_files_2)  # length of list
    print(test_length)

    total_epochs = int(2160*(test_length)) # total length of data array in 20 second epochs


    # establish x and y vectors for input data
    dh1 = display(f'Items left: {test_length}',display_id=True)
    dh = display('Loading...',display_id=True)

    epochs=2160

    y_done=0
    x_done=0

    glob_counter = 0

    for item in (sorted(fourier_files_2)):
        append=''
        x = np.zeros((epochs, 100))

        y = np.zeros((epochs, 1))

        test_length-=1
        dh1.update(f'Items left: {test_length}')

        excl = 0

        item = item.replace("x_ffnorm", "y_ffnorm")

        if os.path.isfile(f'{path_csv}{item}_scored.csv')==True:
          excl=1

        item = item.replace("y_ffnorm", "x_ffnorm")

        if excl==0:
            sub="x_ffnorm"
            ind=item.rfind(sub)
            item_dec=item[ind+2:]
            print(item_dec)
            current_file_x = StandardScaler().fit_transform(np.load(f"{path_Fourier}{item}"))

            x_done=1

            item = item.replace("x_ffnorm", "y_ffnorm")
            if scored > 0:
              current_file_y = np.load(f"{path_Fourier}{item}")
              print(unique(current_file_y))

            y_done=1

            window=3
            shift=window*2
            max_feats=100
            classes=5

            X_score3=np.zeros((len(current_file_x)-shift,window*2+1,max_feats))
            y_score3=np.zeros((len(current_file_x)-shift, classes))

            for i in range(window,len(current_file_x)-window):
              X_score3[i-window]=current_file_x[i-window:i+window+1,0:max_feats]
              # y_score3[i-window]=y[i]
          
            if x_done == 1 and y_done == 1:
                glob_counter+=1

                num_classes=5

                if scored in [0,1]:
                  y_pred=model.predict(X_score3, 2160, verbose=0)

                  y_pred_codes=np.argmax(y_pred,1)

                else:
                  y_pred_codes=current_file_y-1

                y_pred_codes.tofile(f'{path_csv}{item}_{strpart}_scored.csv', sep = ',')

                dh.update(f'loaded x and y for {item}')

                if glob_counter%50==0:
                    print(glob_counter)
                    print((score)/glob_counter)
                x_done=0
                y_done=0



        else:
            print(f'{item} excluded')
            excl=0



In [ ]:
def list_prune_via_substrings(list_to_be_scanned, substring_list):
  result_list=[]

  if type(list_to_be_scanned[0])!=str:
    list_to_be_scanned=str(list_to_be_scanned)

  if type(substring_list[0])!=str:
    substring_list=[str(sub) for sub in substring_list]

  for scan_member in list_to_be_scanned:
    present=1
    for member in substring_list:
      if member in scan_member:
        present=0
    
    if present==1:
      result_list.append(scan_member)

  return result_list

# Load Data (alternate for testing)

In [ ]:
old_cohort = ['566']

held_cohorts = ['200316','566', '569', ' NPM569 ', 'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ',
                'NPM573-576',' NPM592', 'NPM644', 'NPM656', 'NPM652']
wild_types=['NPM648','NPM656', 'NPM652']

wild_types_saline=['NPM644']

saline=[' NPM592 ', ' NPM596 ',' NPM604 ', ' NPM614 ']



# controls = [' NPM592 ',' NPM596 ', ' NPM604 ',' NPM609 ', 'NPM644-647']
recent = [' NPM645 ',' NPM646 ',' NPM647 ',
          'NPM656-659',
          ' NPM661 ',' NPM662 ', ' NPM663 ',
          ' NPM666 ', ' NPM667 ', ' NPM668 ']

extra_train=[' NPM609 ',' NPM644 ', ' NPM664 ']

held_dates = ['200103','200104','200105','200106',
       '200107','200108','200109',
       '200301','200302','200303',
       '200614','200615','200616',
       '200719','200720','200721',
       '200809','200810','200811',
       '201101','200102','201103',
       '201213','201214','201215',
       '210131','200201','210202',
       '210321','200322','210323',
       '200620','200621','200622']


test_data=saline+recent+held_cohorts
load_test_data=0
path=path_Fourier_scored




fourier_files = [f for f in listdir(path) if isfile(join(path, f))]

fourier_files = [f for f in fourier_files if "x_ffnorm" in f]   # find all x arrays in fourier_files


non_train=saline+recent+held_cohorts


print(non_train)
files_held=[]
files_held=[f for d in non_train for f in fourier_files if d in f] # gather list of all non-training files


train_files=[]
train_files=[f for f in fourier_files if f not in files_held] # gather list of all training files



x1,y1=load_Fourier_xy(path, train_files)


saline_files=[]
saline_files=[f for d in saline for f in fourier_files if d in f] # gather list of all saline VGAT Cre files for test dataset
x2,y2=load_Fourier_xy(path, saline_files)

val_files=[]
val_files=[f for d in recent for f in fourier_files if d in f] # gather list of all validation files marked by 'recent' variable
x3,y3=load_Fourier_xy(path, val_files)


[' NPM592 ', ' NPM596 ', ' NPM604 ', ' NPM614 ', ' NPM645 ', ' NPM646 ', ' NPM647 ', 'NPM656-659', ' NPM661 ', ' NPM662 ', ' NPM663 ', ' NPM666 ', ' NPM667 ', ' NPM668 ', '200316', '566', '569', ' NPM569 ', 'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ', 'NPM573-576', ' NPM592', 'NPM644', 'NPM656', 'NPM652']
770


'Items left: 363'

'loaded x and y for y_ffnorm NPM615 200829 191751_094 NPM612-615.npy'

50
100
150
200
250
300
350
400


KeyboardInterrupt: ignored

# Load Data

In [ ]:
# Datasets selected for Train/Test Split via this block

old_cohort = ['566']  # not trained on animals with radically different signal
held_cohorts = ['200316','566', '569', ' NPM569 ', 
                'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ',
                'NPM573-576', ' NPM580 '] # Wild type or control mice with lack of baseline or classifier-breaking scoring issues 

wild_types=['NPM648','NPM652', 'NPM656'] # WT KA mouse cohort identifiers
wild_types_saline=['NPM644'] # WT saline mouse identifiers - no quotes ensures entire 644 cohort is loaded
saline = [' NPM592 ', ' NPM596 ',' NPM604 ', ' NPM614 '] # Saline 


recent = [' NPM645 ',' NPM646 ',' NPM647 ', 'NPM656-659',
          ' NPM661 ',' NPM662 ', ' NPM663 ', ' NPM666 ', 
          ' NPM667 ', ' NPM668 '] # Validation set of most recent animals
          # Mixed batch of KA and Saline, VGAT Cre and WT

extra_train = [' NPM609 ',' NPM644 ', ' NPM664 '] 
# random saline animals (609,644) and recent-KA animals (664) to add to training dataset

held_dates = ['200103','200104','200105','200106',
       '200107','200108','200109',
       '200301','200302','200303',
       '200614','200615','200616',
       '200719','200720','200721',
       '200809','200810','200811',
       '201101','200102','201103',
       '201213','201214','201215',
       '210131','200201','210202',
       '210321','200322','210323',
       '200620','200621','200622']
       # all dates from baseline recordings of KA mice were ignored for training, 
       # due to issues sorting by condition vs. mouse vs. date



path=path_Fourier_scored
fourier_files = [f for f in listdir(path) if isfile(join(path, f))]
fourier_files = [f for f in fourier_files if "x_ffnorm" in f]   # find all x arrays in fourier_files

non_train=saline+wild_types_saline+recent+held_cohorts+wild_types

print(non_train)
files_held=[]
files_held=[f for d in non_train for f in fourier_files if d in f] # gather list of all non-training files


train_files=[]
train_files=[f for f in fourier_files if f not in files_held] # gather list of all training files



x1,y1=load_Fourier_xy(path, train_files)


saline_files=[]
saline_files=[f for d in saline for f in fourier_files if d in f] # gather list of all saline VGAT Cre files for test dataset
x2,y2=load_Fourier_xy(path, saline_files)

val_files=[]
val_files=[f for d in recent+wild_types for f in fourier_files if d in f] # gather list of all validation files marked by 'recent' variable
x3,y3=load_Fourier_xy(path, val_files)


[' NPM592 ', ' NPM596 ', ' NPM604 ', ' NPM614 ', 'NPM644', ' NPM645 ', ' NPM646 ', ' NPM647 ', 'NPM656-659', ' NPM661 ', ' NPM662 ', ' NPM663 ', ' NPM666 ', ' NPM667 ', ' NPM668 ', '200316', '566', '569', ' NPM569 ', 'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ', 'NPM573-576', ' NPM580 ', 'NPM648', 'NPM652', 'NPM656']
719


'Items left: 0'

'loaded x and y for y_ffnorm NPM665 210622 070057_015 NPM665-668.npy'

50
100
150
200
250
300
350
400
450
500
550
600
650
700
145


'Items left: 0'

'loaded x and y for y_ffnorm NPM614 200811 191015_058 NPM612-615.npy'

50
100
323


'Items left: 0'

'loaded x and y for y_ffnorm NPM668 210713 190351_058 NPM665-668.npy'

50
100
150
200
250
300


In [ ]:
wild_type_list=[f for d in wild_types+wild_types_saline for f in fourier_files if d in f]

names=[]
for i in wild_type_list:
  names.append(i[5:11])

print(len(unique(names)))

saline_list=[f for d in saline+wild_types_saline+[' NPM609 '] for f in fourier_files if d in f]

names=[]
for i in saline_list:
  names.append(i[5:11])

print(len(unique(names)))


# non_KA_list=saline+wild_types_saline+held_cohorts+[' NPM609 ']
# non_KA=[]
# non_KA=[f for d in non_KA_list for f in fourier_files if d in f] # gather list of all non-training files
# train_files=[]
# KA_list=[f for f in fourier_files if f not in non_KA] # gather list of all training files

path=path_Fourier_scored
fourier_files = [f for f in listdir(path) if isfile(join(path, f))]
fourier_files = [f for f in fourier_files if "x_ffnorm" in f]   # find all x arrays in fourier_files

non_train=saline+wild_types_saline+recent+held_cohorts+wild_types

print(non_train)
files_held=[]
files_held=[f for d in non_train for f in fourier_files if d in f] # gather list of all non-training files


train_files=[]
train_files=[f for f in fourier_files if f not in files_held] # gather list of all training files

names=[]
for i in train_files:
  names.append(i[9:15])

l=unique(names)
print(*l, sep = "\n")
print(len(unique(names)))

names=[]
saline_files=[f for d in saline for f in fourier_files if d in f] # gather list of all saline VGAT Cre files for test dataset
for i in saline_files:
    names.append(i[9:15])

print(*unique(names), sep = "\n")
print(len(unique(names)))



names=[]
val_files=[f for d in recent+wild_types for f in fourier_files if d in f] 
for i in val_files:
    names.append(i[9:15])

print(*unique(names), sep = "\n")
print(len(unique(names)))

# Model Creation

In [ ]:
# create array of codes to link numeric labels in y variables to state labels

codes=[None]*5
codes[0]='wake'
codes[1]='nrem' 
codes[2]='rem'
codes[3]='seizure'
codes[4]='post-ictal'

# global variables for future functions
i=16
full=1

In [ ]:
keras.backend.clear_session()

In [ ]:
# run before re-running training


X_train_seq=None
X_test_seq=None
X_held_seq=None
model=None
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()
# tf.reset_default_graph()

for i in range(0,50):
  gc.collect()


In [ ]:
gc.collect()

%tensorboard --logdir f"./Logs/"


# Pre-Process X and Y Variables


In [ ]:
# These arrays correspond to lists of indices in the feature vector
# for each of our channels

mask_DTRMS=np.concatenate(([12,19,32,39,52,59,72,79],np.arange(80,100)))
print(len(mask_DTRMS))

mask_FFT_only = []
for i in [0,1,2,3]:
  ind1=(i*20)+6
  ind2=(i*20)+13
  mask_FFT_only=np.concatenate((mask_FFT_only,np.arange(ind1,ind2)))

mask_FFT_only=np.concatenate((mask_FFT_only,np.arange(80,100)))
mask_FFT_only=[int(i) for i in mask_FFT_only]
print(len(mask_FFT_only))


mask_ECoG=np.arange(0,20)
mask_EMG=np.arange(20,40)
mask_HPCL=np.arange(40,60)
mask_HPCR=np.arange(60,80)
mask_RMS=np.arange(80,100)

mask_FullFeats=np.arange(0,100)

# Designate feature mask to use for preparing variables
# This construction would send just "ECoG" channel to training

# mask=np.concatenate((mask_ECoG)) 
mask = np.concatenate((mask_HPCL, mask_HPCR)) # mask_FullFeats lets all 100 features through


28
48


In [ ]:
print(mask_FFT_only)

[6, 7, 8, 9, 10, 11, 12, 26, 27, 28, 29, 30, 31, 32, 46, 47, 48, 49, 50, 51, 52, 66, 67, 68, 69, 70, 71, 72, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [ ]:
# Convert ys to one-hot labels, store x variables to 
# counteract future edits-in-place

X_train=x1
y_train=np_utils.to_categorical(y1, num_classes=5)

X_test=x2
y_test=np_utils.to_categorical(y2, num_classes=5)

X_held=x3
y_held=np_utils.to_categorical(y3, num_classes=5)

X_train_masked=X_train[:,mask]
X_test_masked=X_test[:,mask]
X_held_masked=X_held[:,mask]

In [ ]:
X_train_masked.shape

# SVM Evaluation

In [ ]:
from IPython.lib.display import isdir
from sklearn.svm import LinearSVC
import sklearn
mu=.15
class_weight=norm_sklearn_classweight(y_train, mu=mu)

savepath=f'./Results/CM_SVM/'

if os.path.exists(savepath)==0:
  os.mkdir(savepath)


results=np.zeros((3,5,3))
feature_set_count=0
for masks in [[mask_DTRMS,'SVM: D/T Ratio & RMS EMG', 'DTRMS'],
              [mask_FFT_only, 'SVM: FFT, D/T \n Ratio & RMS EMG', 'FFT'],
               [mask_none, 'SVM: Full Features', 'FullFeats']]:

  print(f'\n{masks[1]}')
  X_train_masked=X_train[:,masks[0]]
  X_test_masked=X_test[:,masks[0]]
  X_held_masked=X_held[:,masks[0]]

  svm_model = sklearn.linear_model.SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, 
                                                      fit_intercept=True, max_iter=1000, tol=0.001, shuffle=False, verbose=0, epsilon=0.1, n_jobs=None, 
                                                      random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, 
                                                      n_iter_no_change=5, class_weight=class_weight, warm_start=False, average=False)

  svm_model.fit(X_train_masked, np.argmax(y_train,1))

  
  for x, y, name in [[X_train_masked, y_train, ' Training Dataset'], [X_test_masked, y_test, ' Saline Control Dataset'], [X_held_masked, y_held, ' Saline-KA Validation Dataset']]:
    y_pred=svm_model.predict(x)
    
    print_conf_mat(np.argmax(y,1), y_pred, masks[1]+' \n '+name, output_name=savepath+masks[2]+name)

    y_pred_codes=y_pred
    y_test_codes=np.argmax(y,1)

    print(classification_report(y_test_codes, y_pred_codes))

    dict1 = classification_report(y_test_codes, y_pred_codes, output_dict=True)
    df = pd.DataFrame(data=dict1)
    df = (df.T)
    print (df)

    df.to_excel(f'{savepath+masks[2]+name}.xlsx')


  for state_count in range(0,5):
    
    discriminant=np.argmax(y_train,1)==state_count
    if True in discriminant:
      results[feature_set_count,state_count,0] = svm_model.score(X_train_masked[discriminant], np.argmax(y_train,1)[discriminant])
    
    discriminant=np.argmax(y_test,1)==state_count
    if True in discriminant:
      results[feature_set_count,state_count,1] = svm_model.score(X_test_masked[discriminant], np.argmax(y_test,1)[discriminant])

    discriminant=np.argmax(y_held,1)==state_count
    if True in discriminant:
      results[feature_set_count,state_count,2] = svm_model.score(X_held_masked[discriminant], np.argmax(y_held,1)[discriminant])

  
  print("\n")
  feature_set_count+=1


In [ ]:
  for x, y, name in [[X_train_masked, y_train, ' Training Dataset'], [X_test_masked, y_test, ' Saline Control Dataset'], [X_held_masked, y_held, ' Saline-KA Validation Dataset']]:
    y_pred=svm_model.predict(x)
    
    print_conf_mat(np.argmax(y,1), y_pred, masks[1]+' \n '+name, output_name=savepath+masks[2])

    y_pred_codes=y_pred
    y_test_codes=np.argmax(y,1)

    print(classification_report(y_test_codes, y_pred_codes))

    dict1 = classification_report(y_test_codes, y_pred_codes, output_dict=True)
    df = pd.DataFrame(data=dict1)
    df = (df.T)
    print (df)

    df.to_excel(f'{savepath+masks[2]+name}.xlsx')

In [ ]:
import matplotlib.patches as mpatches
plt.rcParams['figure.figsize']=[10,10]

red_patch = mpatches.Patch(color='r', label='D/T Ratio + RMS EMG')
green_patch = mpatches.Patch(color='g', label='FFT Magn. only')
blue_patch = mpatches.Patch(color='b', label='Full Feature Set w PSDs')

for i in range(3): 
  shape_patch_1=matplotlib.lines.Line2D([],[],color=(0,0,0),marker='o', label='Train')
  shape_patch_2=matplotlib.lines.Line2D([],[],color=(0,0,0),marker='x', label='Test')
  shape_patch_3=matplotlib.lines.Line2D([],[],color=(0,0,0),marker='<', label='Validation')


for i in range(3):
  marker=['o','x','<'][i]
  for c in range(0,5):
    plt.scatter(['W','N','R','Sz','P'][c],results[0,c,i].T, marker=marker, color=(1,0,0), s=300, alpha=.6)
    plt.scatter(['W','N','R','Sz','P'][c],results[1,c,i].T, marker=marker, color=(0,1,0), s=300, alpha=.6)
    plt.scatter(['W','N','R','Sz','P'][c],results[2,c,i].T, marker=marker, color=(0,0,1), s=300, alpha=.6)

plt.ylim([0,1])
plt.ylabel('Accuracy')
plt.title('SVM Performance')
plt.legend(handles=[red_patch, green_patch, blue_patch, shape_patch_1, shape_patch_2, shape_patch_3], bbox_to_anchor=(1.02, 1),
                  loc='upper left', borderaxespad=0.)

    

In [ ]:
len(np.argmax(y_train,1)==i)


# Full Deep Learning Grid Search

In [ ]:
mask_ECoG=np.arange(0,20)
mask_EMG=np.arange(20,40)
mask_HPCL=np.arange(40,60)
mask_HPCR=np.arange(60,80)
mask_RMS=np.arange(80,100)

Channel_Masks=[[mask_ECoG, 'ECoG_only'],
               [np.concatenate((mask_HPCL,mask_HPCR)), '2xHPC'],
               [np.concatenate((mask_EMG,mask_HPCL,mask_HPCR,mask_RMS)), 'no_ECoG'],
               [np.concatenate((mask_EMG,mask_RMS)), 'EMG_RMS_only' ],
               [np.concatenate((mask_ECoG,mask_EMG,mask_RMS)), 'noHPC']]

Feature_Masks = [[mask_DTRMS,'DTR+RMS only'],
  [mask_FFT_only, 'FFT only'],
  [mask_FullFeats, 'FullFeats']]

for window_length in [0,1,2,3]:
  # Test all four windowing variants 
  # Grid Search Iteration Count in This Loop: 4 Combinations
  for masks in Feature_Masks:
    # Test all three Feature Vectors 
    # Grid Search Iteration Count in This Loop: 12 Combinations

    print(f'\n{masks[1]}')
    X_train_masked=X_train[:,masks[0]]
    X_test_masked=X_test[:,masks[0]]
    X_held_masked=X_held[:,masks[0]]

    submodel=masks[1]

    window_length=window_length # number of time points on either side of scored epoch to use
    # for input variables.  i.e. window_length 3 leads to an input 7 epochs long

    max_feats=X_train_masked.shape[1]

    X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_held_seq=generate_sequences(X_held_masked, windows=window_length, x_or_y='X', max_feats=max_feats)

    y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_held_seq=generate_sequences(y_held, windows=window_length, x_or_y='Y', max_feats=max_feats)

    print(X_train_seq.shape)

    # Establishing variables for training
    layers=1 # number of Bi-LSTM layers
    nn=200 # target neuron count
    epochs=20 # number of epochs to train 
    mu=.15 # mu factor of modified sklean_class_weight function


    print(mu)
    class_weight=norm_sklearn_classweight(y_train_seq, mu=mu)

      
    print(class_weight)

    for nn in [50,100,200]:
      # Test Three Initial-Layer Neuron Counts
      # Grid Search Iteration Count in This Loop: 36 Combinations

      # Test All Above Over 4 Types of Models: 144 Total Model Variants


      gc.collect()

      model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      gc.collect()

      savepath='./Results/BiLSTM/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      

      model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=4, basepath=basepath)
      gc.collect()
      savepath='./Results/Quad_BiLSTM/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'quad-layer_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      
      
      model, model_history, logpath, csvpath, model_name = create_train_LSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      gc.collect()
      savepath='./Results/LSTM/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      
    
    
      # model, model_history, logpath, csvpath, model_name = create_train_CNN_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
      #                         epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      # gc.collect()
      # savepath='./Results/CNN/'


      # save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
      #             model_name=model_name+f'_{submodel}_win{window_length}', 
      #             model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      


      # model, model_history, logpath, csvpath, model_name = create_train_flat_LSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
      #                         epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      # gc.collect()
      # save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
      #             model_name=model_name+f'_{submodel}_win{window_length}', 
      #             model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      


      model, model_history, logpath, csvpath, model_name = create_train_Dense_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      gc.collect()
      savepath='./Results/Dense/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'_{submodel}_win{window_length}', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)



# Evaluate Top Models to 60 Epochs

In [ ]:
mask_ECoG=np.arange(0,20)
mask_EMG=np.arange(20,40)
mask_HPCL=np.arange(40,60)
mask_HPCR=np.arange(60,80)
mask_RMS=np.arange(80,100)

Channel_Masks=[[mask_ECoG, 'ECoG_only'],
               [np.concatenate((mask_HPCL,mask_HPCR)), '2xHPC'],
               [np.concatenate((mask_EMG,mask_HPCL,mask_HPCR,mask_RMS)), 'no_ECoG'],
               [np.concatenate((mask_EMG,mask_RMS)), 'EMG_RMS_only' ],
               [np.concatenate((mask_ECoG,mask_EMG,mask_RMS)), 'noHPC']]

Feature_Masks = [[mask_DTRMS,'DTR+RMS only'],
  [mask_FFT_only, 'FFT only'],
  [mask_FullFeats, 'FullFeats']]

for window_length in [3]:

  for masks in [Feature_Masks[2]]:

    print(f'\n{masks[1]}')
    X_train_masked=X_train[:,masks[0]]
    X_test_masked=X_test[:,masks[0]]
    X_held_masked=X_held[:,masks[0]]

    submodel=masks[1]

    window_length=window_length # number of time points on either side of scored epoch to use
    # for input variables.  i.e. window_length 3 leads to an input 7 epochs long

    max_feats=X_train_masked.shape[1]

    X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_held_seq=generate_sequences(X_held_masked, windows=window_length, x_or_y='X', max_feats=max_feats)

    y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_held_seq=generate_sequences(y_held, windows=window_length, x_or_y='Y', max_feats=max_feats)

    print(X_train_seq.shape)

    # Establishing variables for training
    layers=1 # number of Bi-LSTM layers
    nn=200 # target neuron count
    epochs=60 # number of epochs to train 
    mu=.15 # mu factor of modified sklean_class_weight function


    print(mu)
    class_weight=norm_sklearn_classweight(y_train_seq, mu=mu)

      
    print(class_weight)

    for nn in [50,100,200]:

      gc.collect()

      model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      gc.collect()

      savepath='./Results/BiLSTM/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      

      model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=4, basepath=basepath)
      gc.collect()
      savepath='./Results/Quad_BiLSTM/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'quad-layer_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      
      
      model, model_history, logpath, csvpath, model_name = create_train_LSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      gc.collect()
      savepath='./Results/LSTM/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      
    



# Train Interpretable Machine Learning Models via Channel-Dropping

In [ ]:
mask_ECoG=np.arange(0,20)
mask_EMG=np.arange(20,40)
mask_HPCL=np.arange(40,60)
mask_HPCR=np.arange(60,80)
mask_RMS=np.arange(80,100)

Channel_Masks=[[mask_ECoG, 'ECoG_only'],
               [np.concatenate((mask_HPCL,mask_HPCR)), '2xHPC'],
               [np.concatenate((mask_EMG,mask_HPCL,mask_HPCR,mask_RMS)), 'no_ECoG'],
               [np.concatenate((mask_EMG,mask_RMS)), 'EMG_RMS_only' ],
               [np.concatenate((mask_ECoG,mask_EMG,mask_RMS)), 'noHPC']]

Feature_Masks = [[mask_DTRMS,'DTR+RMS only'],
  [mask_FFT_only, 'FFT only'],
  [mask_FullFeats, 'FullFeats']]

for window_length in [3]:

  for masks in Channel_Masks:

    print(f'\n{masks[1]}')
    X_train_masked=X_train[:,masks[0]]
    X_test_masked=X_test[:,masks[0]]
    X_held_masked=X_held[:,masks[0]]

    submodel=masks[1]

    window_length=window_length # number of time points on either side of scored epoch to use
    # for input variables.  i.e. window_length 3 leads to an input 7 epochs long

    max_feats=X_train_masked.shape[1]

    X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_held_seq=generate_sequences(X_held_masked, windows=window_length, x_or_y='X', max_feats=max_feats)

    y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_held_seq=generate_sequences(y_held, windows=window_length, x_or_y='Y', max_feats=max_feats)

    print(X_train_seq.shape)

    # Establishing variables for training
    layers=1 # number of Bi-LSTM layers
    nn=200 # target neuron count
    epochs=60 # number of epochs to train 
    mu=.15 # mu factor of modified sklean_class_weight function


    print(mu)
    class_weight=norm_sklearn_classweight(y_train_seq, mu=mu)

      
    print(class_weight)

    for nn in [200]:

      gc.collect()

      model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_test_seq, y_train_seq, y_test_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, n_iter=nn, batch_factor=.001, layers=1, basepath=basepath)
      gc.collect()

      savepath='./Results/Interpretable/'

      save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                            model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath, savepath=savepath)
      



# Generate Classification Matrices for Previous Models

In [ ]:
# This block can be uncommented to quickly plot existing model sizes
mask=mask_FullFeats # mask_none lets all 100 features through
submodel="FullFeats"

X_train=X_train2[:,mask]
X_test=X_test2[:,mask]
X_held=X_held2[:,mask]

window_length=3 # number of time points on either side of scored epoch to use
# for input variables.  i.e. window_length 3 leads to an input 7 epochs long

X_train_seq=generate_sequences(X_train, windows=window_length, x_or_y='X', max_feats=100)
X_test_seq=generate_sequences(X_test, windows=window_length, x_or_y='X', max_feats=100)
X_held_seq=generate_sequences(X_held, windows=window_length, x_or_y='X', max_feats=100)

y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=100)
y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=100)
y_held_seq=generate_sequences(y_held, windows=window_length, x_or_y='Y', max_feats=100)

# model_folder='/content/drive/MyDrive/Classifier Final/Pretrained/Models/'

# model_path=f'{model_folder}_Model_LSTM_size_25_full_20230110-215403800.h5'
# model=load_model(model_path)
# class_matrix(model, ctrls=2, run=2, x=None, y=None, model_name=f' {submodel}_ctrl_train_stdscaler_fftrmsemg_win{window}', model_num=0, LSTM_size=25, basepath=basepath)

model_path='/content/drive/MyDrive/Classifier Final/Results/Final Model/Final Model BiLSTM200.h5'
model=load_model(model_path)
model.summary()
submodel='BiLSTM_200'

save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, 
                      X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, 
                      y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                      model_name=model_name+f'_{submodel}_win{window_length}', 
                      model_num=nn, basepath=basepath, logpath=logpath, csvpath=csvpath)

# R&K Scoring Layer

In [ ]:
model=load_model(f'{basepath}/Results/Final Model/Final Model BiLSTM200.h5', compile=False)
model.compile()

model.summary()

agree_sum=[]
dh1 = display(f'Item: {0}',display_id=True)

arr=X_test

for i in range(int(len(arr)/2160)-1):
  dh1.update(f'Item: {i}')
  X_array_seq=[]
  X_array_seq=arr[i*2160:(i+1)*2160,:]

  if len(X_array_seq.shape)<3:
    X_array_seq=generate_sequences(X_array_seq, 3, x_or_y='X', max_feats=X_array_seq.shape[1])

  # print(X_array_seq.shape)

  y_pred=model.predict(X_array_seq, batch_size=2160,verbose=0);
  y_pred_scores = np.argmax(y_pred,1)

  y_pred_scores_baseline = y_pred_scores

  # Rechtshaffen and Kales Critera

  for idx in range(0,len(y_pred_scores)-1):
    if idx>0:
      # REM Must Follow NREM
      # "If REM is preceded by Wake, score as Wake"
      if y_pred_scores[idx]==2 and y_pred_scores[idx-1]==0:
        y_pred_scores[idx]=0

      # In order to score NREM, there must be 2 consecutive epochs
      # or lone NREM is scored as the previous epoch
      if y_pred_scores[idx]==1:
        if y_pred_scores[idx-1]!=1 and y_pred_scores[idx+1]!=1:
          y_pred_scores[idx]=y_pred_scores[idx-1]

      # In order to score Wake, there must be 2 consecutive epochs
      # or lone Wake is scored as the previous epoch
      if y_pred_scores[idx]==0:
        if y_pred_scores[idx-1]!=0 and y_pred_scores[idx+1]!=0:
          y_pred_scores[idx]=y_pred_scores[idx-1]
      
  agreement = y_pred_scores == y_pred_scores_baseline

  agree_pct=len(agreement[agreement==True])/len(agreement)
  # print(len(agreement[agreement==False]))
  # print(agree_pct)
  agree_sum.append(agree_pct)


print(np.mean(agree_sum))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 7, 400)           481600    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 7, 400)            0         
                                                                 
 flatten_1 (Flatten)         (None, 2800)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 14005     
                                                                 
 activation_1 (Activation)   (None, 5)                 0         
                                                                 
Total params: 495,605
Trainable params: 495,605
Non-trainable params: 0
________________________________________________

'Item: 143'

1.0


(array([], dtype=int64),)

# Final Scoring of Data

In [ ]:
model=load_model('/content/drive/MyDrive/Classifier Final/Results/Final Model/Final Model BiLSTM200.h5')
model.compile()

model.summary()

held_cohorts = []


# held_dates=held_cohorts+controls+held_dates
held_dates=[]

scoretypes=[[0,'unscored'],[1,'class_scored'],[2,'expert_scored']]

errors=[]

for flag, types in scoretypes:
  print(f'Generating CSVs for {types}')
  score_data(path, errors, load_dropped_data=0, held_dates=[], scored=flag)


# Process CSVs into Figures

In [ ]:
csv_folder='/content/drive/MyDrive/Classifier Final/CSV_Outputs/'
path_csv=f'{csv_folder}csv_classifier_prev_un_scored/'
path_holes=f'{csv_folder}csv_trainset_classifier_scored/'
mouse_sort_csvs=f'{csv_folder}csv_full/'
path_expert=f'{csv_folder}csv_trainset_expert_scored/'
path_figs=f'{csv_folder}figures/'

for folder in [path_csv, path_holes, mouse_sort_csvs, path_expert, path_figs]:
  if os.path.exists(folder)==0:
    os.mkdir(folder)

In [ ]:
for sel in [0,1,2]:

  filepath = [path_csv,path_holes,path_expert][sel]
  filelist=os.listdir(filepath)
  print(len(filelist))
  print(sorted(filelist))
  count=0
  if count==0:
    dh1 = display(f'Items left: {len(filelist)}',display_id=True)

  for item in sorted(filelist)[::-1]:
    if '.csv' in item:

      count+=1    
      
      if count%50==0 or (len(filelist)-count)==0:
        dh1.update(f'Items left: {len(filelist)-count}')

      ## Extract Mouse Name
      ind_name = item.find('NPM')
      name = item[ind_name:ind_name+6]
      
      path_out=f'{mouse_sort_csvs}{name}/'

      if os.path.exists(path_out)==0:
        os.mkdir(path_out)

      if os.path.isfile(f'{path_out}{item}')==0:
        shutil.copy(f'{filepath}{item}', f'{path_out}{item}')

## Plot Agreement Between Expert and Classifier

In [ ]:
# from re import A
plt.rcParams['figure.figsize'] = [10, 5]
import matplotlib.style as mplstyle
mplstyle.use('fast')



folderlist=sorted(os.listdir(mouse_sort_csvs))
folderlist=[f for f in sorted(os.listdir(mouse_sort_csvs)) if '.csv' not in f]
print(folderlist)


use_expert=0

if use_expert==1:
  skipped_type='classifier_scored'
  use_type='expert_scored'

  score_label='Expert Scoring'
else:
  skipped_type='expert_scored'
  use_type='classifier_scored'

  score_label='Trained or Tested Data'

# print(folderlist)

KA_list=[605,606,607,608,610,611,612,
         613,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,642,648,649,650,651,652,653,
         654,655,656,657,658,659,661,662,663,664,665,666,667,668,688,690,691,693,695,696,697]

Excluded=[554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,573,576,577,578,579,589,591,593,594,595,580,605,606,607,
          608,609,610,611,612,613,614,615,616,617,618,630,631,635,637,641,642,643,648,651,655,657,658,662,663]
# Controls: 644,645,646,647,
slope_sum=np.zeros((4,1))
avg_slope=np.zeros((4,1))

n_animals=0

Controls=True



for folder in folderlist[-22::-1]:
  
  Control=0
  KA_flag=False
  skip=0
  bad_flag=0
  Sz_occur=0
  mouse_folder = f'{mouse_sort_csvs}{folder}/'
  filelist=sorted(os.listdir(mouse_folder))

  count_state = np.zeros((len(filelist),6))
  count_state_total = np.zeros((len(filelist),5))
  cumu_sum = np.zeros((len(filelist),5))

  if os.path.isdir(f'{path_figs}{folder}')==False:
    os.mkdir(f'{path_figs}{folder}')

  filelist = [f for f in filelist if skipped_type in f]   
  for KA in KA_list:
    if f'{KA}' in folder:
      KA_flag=True

  for Excl in Excluded:
    if f'{Excl}' in folder:
      skip=1

  if KA_flag!=Controls and skip==0:
    print(folder)
    sz=0
    total = len(filelist)
    # print(total)
    arr = np.zeros((total,2))
    count=0

    if len(filelist)>0:
      for item in filelist[::-1]:
        if use_type in item:
           count_state[count][5]=1
        print(item)
        path = open(f'{mouse_folder}{item}')

        if 'expert_scored' in item:
          array2 = np.loadtxt(path, delimiter=",",dtype='float')
          array2=array2.astype('int')

        item2=item.replace("expert", "classifier")

        # print(item2)
        path = open(f'{mouse_folder}{item2}')

        # print(item)
        if 'expert_scored' not in item2:
          array = np.loadtxt(path, delimiter=",",dtype='int')   

        path.close()

        name_ind=item.find('NPM')
        time_ind=name_ind+14
        if item[time_ind:time_ind+2]=='19':
          Z_time='night'
        else:
          Z_time='day'

        # print(len(array2))

        x1 = np.arange(0,2160)
        
        width=16
        height=width/4
        plt.figure(figsize=(width,height), dpi=2000)
        ax1=plt.subplot(2,4, (1,3))  

        x_legend_offset=-.1

        agreement = array==array2[3:2160-3]

        agreement_score=np.zeros(len(agreement))
        agreement_score[agreement==True]=1
        agreement_score[agreement==False]=0

        agree_pct=len(agreement[agreement==True])/(2160-6)
        # print(len(agreement[agreement==False]))
        print(agree_pct)
# Ax1
        
        ax1.bar(x1[3:2160-3], agreement_score,color='k', width=1)
        ax1.bar(np.ma.masked_where(~agreement, x1[3:2160-3]), np.ma.masked_where(agreement, agreement_score+1), color='grey', width=1)

        # ax1.title.set_text(f'Sleep Record Scoring Comparision')

        black_patch = mpatches.Patch(color='k', label='Agreement')
        grey_patch = mpatches.Patch(color='grey', label='Difference')

        ax1.legend(handles=[black_patch, grey_patch], bbox_to_anchor=(1.02, 1),
                         loc='upper left', borderaxespad=0.)
        ax1.text(x_legend_offset, .5, f'Classifier and \n Expert \n Agreement', horizontalalignment='center', 
                 verticalalignment='center', transform=ax1.transAxes)
        ax1.set_xticks(range(0,2160+int(2160/12),int(2160/12)))

        if Z_time=='night':
          time_vect=[str(x)+":00" for x in list(range(19,24))+list(range(0,8))]
        else:
          time_vect=[str(x)+":00" for x in range(7,20)]


        ax1.set_xticklabels(time_vect)

# Ax2

        ax2=plt.subplot(2,4,(5,7))  

        codes=['PI', 'Sz', 'W', 'N', 'R']
        types=max(array)
        # print(f'max legend = {types}')        
        
        array[array==4]=-2
        array[array==3]=-1

        array2[array2==4]=-2
        array2[array2==3]=-1

        ax2.plot(x1[3:2160-3], array, 'k')
        ax2.plot(x1[3:2160-3], array2[3:2160-3], color='grey')

# agreement plotting

        grey_patch = mpatches.Patch(color='grey', label='Expert Scoring')
        black_patch = mpatches.Patch(color='k', label='Classifier Scoring')
        ax2.legend(handles=[black_patch, grey_patch], bbox_to_anchor=(1.02, 1),
                         loc='upper left', borderaxespad=0.)

        ax2.text(x_legend_offset, .5, f'Hypnogram', horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
        ax2.set_xticks(range(0,2160+int(2160/12),int(2160/12)))
        ax2.set_xticklabels(time_vect)
        ax2.set_xlabel("Time")
        if Z_time=='night':
          ax2.set_facecolor("lightgray")

# y tick assignment      
        
        ticklist=ax2.get_yticks()
        # print(ticklist)
        low_legend=min([min(array), min(array2)])
        # print(f'low legend = {low_legend}')        

        high_legend=max([max(array), max(array2)])
        # print(f'high legend = {high_legend}')



        ax2.set_yticks(range(-2,3))
        ax2.set_yticklabels(labels=codes)
        ax2.invert_yaxis()        


# Ax3
        ax3=plt.subplot(2,4, (4))
        ax3.pie([agree_pct,1-agree_pct], colors=['k', 'grey'],autopct='%1.1f%%', pctdistance=1.7, center=(0,-5))
        pie = ax3.get_position()
        pie.y0 = pie.y0 - 0.05
        pie.y1 = pie.y1 - 0.05

        ax3.set_position(pie)
                


        # plt.plot(x1[3:2160-3], array, 'b--')
        # plt.plot(x1, array2, 'r-')
        # plt.plot(x1[3:2160-3], agreement_score)
        plt.show()

        figname=f'{path_figs}{folder}/{item}_agreement.jpg'
        plt.savefig(figname, bbox_inches='tight')

        count+=1

      

## Kaplan-Meier Seizure Graph


In [ ]:
# from re import A
plt.rcParams['figure.figsize'] = [10, 5]

folderlist=sorted(os.listdir(mouse_sort_csvs))
folderlist=[f for f in sorted(os.listdir(mouse_sort_csvs)) if '.csv' not in f]
print(folderlist)

max_disp_files=38

use_expert=1

if use_expert==1:
  skipped_type='classifier_scored'
  use_type='expert_scored'

  score_label='Expert Scoring'
else:
  skipped_type='expert_scored'
  use_type='classifier_scored'

  score_label='Trained or Tested Data'

# print(folderlist)

KA_list=[605,606,607,608,610,611,612,
        613,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,642,648,649,650,651,652,653,
        654,655,656,657,658,659,661,662,663,664,665,666,667,668,688,690,691,693,695,696,697]

KA_list_pre_cannula_move=[554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,573,575,576,577,578,579,580,590,589,590,591,593,594,595,605,606,607,
          608,610,611,612,613,615,616,617,618]

Excl_KA=[568,573,577,578,591,594,616,618,619,623,627,630,631,637,641,648,649,650,651,652,653,654,655,656,657,658,659,660,662,665]
Excluded=[630,631,637,641,643,648,651,655,657,658,662,663]

KA_list=KA_list+ KA_list_pre_cannula_move
Excluded=Excl_KA+Excluded

for Controls in [False]:

  folderlist=list_prune_via_substrings(folderlist, Excluded)
  if Controls==True:
    target_folders=list_prune_via_substrings(folderlist, KA_list)

  if Controls==False:
    Control_folders=list_prune_via_substrings(folderlist, KA_list)
    target_folders=list_prune_via_substrings(folderlist, Control_folders)

  slope_sum=np.zeros((4,1))
  avg_slope=np.zeros((4,1))
  
  num_mice=len(target_folders)    
  aggregate_sz=np.zeros((num_mice,max_disp_files))

  n_animals=0

  for folder in target_folders[:]:
    Control=False
    skip=0
    bad_flag=0
    KA_flag=False
    Sz_occur=0

 

    mouse_folder = f'{mouse_sort_csvs}{folder}/'
    filelist=sorted(os.listdir(mouse_folder))

    filelist = [f for f in filelist if skipped_type not in f]   
    print(folder)

    sz=0
    total = len(filelist)
    arr = np.zeros((total,2))
    count=0


    name_ind=filelist[0].find('NPM')

    date_ind=name_ind+7
    date_record=int(filelist[0][date_ind:date_ind+6])
    if date_record < 191206:
      injection_file=0*2
    elif date_record < 200217:
      injection_file=34*2
    elif date_record < 200608:
      injection_file=16*2
    else:
      injection_file=9*2

    filelist=filelist[injection_file:injection_file+max_disp_files]


    count_state = np.zeros((len(filelist),6))
    count_state_total = np.zeros((len(filelist),5))
    cumu_sum = np.zeros((len(filelist),5))

    if len(filelist)>0:
      print(len(filelist))
      for item in filelist[:]:
        if use_type in item:
          count_state[count][5]=1
        path = open(f'{mouse_folder}{item}')
        if 'expert_scored' not in item:
          array = np.loadtxt(path, delimiter=",",dtype='int')

        if 'expert_scored' in item:
          array = np.loadtxt(path, delimiter=",",dtype='float')
          array=array.astype('int')

        ### Counting Loop
        first_inds=[]
        for i in range(5):
            ind_pres=np.where(array==i)[0]
            grp_s_ind, inds=consecutive(ind_pres)

            first_s=[]

            for i in range(len(grp_s_ind)):
                first_s.append(grp_s_ind[i])

            first_s=np.asarray(first_s, order='K')
            first_inds.append((first_s))
        codes=unique(array)
        if 3 in codes:
          Sz_occur=1
        
        for state in range(5):
          statecount=np.zeros((len(first_inds[state]),1))
          for state_burst in range(len(first_inds[state])):
            count_state_total[count][state]=count_state_total[count][state]+(len(first_inds[state][state_burst]))

          count_state[count][state]=len(first_inds[state])-1

          if count==0:
            cumu_sum[count][state]=count_state[count][state]
            



          else:
            cumu_sum[count][state]=cumu_sum[count-1][state]+count_state[count][state]          
            
          if state==3:
            if count <= max_disp_files:
              aggregate_sz[n_animals,count]+=cumu_sum[count][state]

        count+=1
    n_animals+=1
        
      #######    
      
  fig, ax = plt.subplots()
  axes = [ax, ax.twinx()]

  for i in range(n_animals):
    xmax=60
    ymax=300
    ymax2=60
    tick_marks=14

    # n_animals+=1
    ax4 = axes[-1]
    x1 = np.arange(0,len(filelist))

    # scored=count_state[:,5]==1     
    # unscored=count_state[:,5]==0

    scored=count_state[:,5]==1     
    unscored=count_state[:,5]==0

    scoredstyle='k'
    scoredstyle=[np.random.rand(),np.random.rand(),np.random.rand()]
    unscoredstyle='r'
    x_legend_offset=-.15

    # if n_animals==1:
    #   
    # ax4 = plt.subplot(111)
    # sz_ratio=cumu_sum[:,3]
    # y=count_state[:,3]
    y=np.log(aggregate_sz[i,:])
    # ax4.plot(sz_ratio)

    # print(len(count_state))
    # print(len(scored))
    # print(len(unscored))
    # print(len(x1))
    # print(len(y))
    
    if len(scored)>0:
      # ax4.plot(np.ma.masked_where(scored, x1), np.ma.masked_where(scored, y), 'r')
      ax4.plot(x1,y,c=scoredstyle)
    # if len(unscored)>0:
    #   ax4.plot(np.ma.masked_where(unscored, x1), np.ma.masked_where(unscored, y),unscoredstyle)
    # ax4.set(xlim=(0,xmax))
    # ax4.set(ylim=(0,20))
    ax4.grid(True,'minor','x',markevery=tick_marks)
    # ax4.set_yscale('log')

    # ax5 = plt.subplot(515)
    # ax5.scatter(sum(count_state[:,3]), sum(count_state[:,0]))
    # ax5.set(xlim=(0,xmax))
    # # ax5.set(ylim=(0,ymax2))
    # ax5.grid(True,'major','x',markevery=tick_marks)

    # ax5.set_yscale('log') 
    red_patch = mpatches.Patch(color='red', label=score_label)

    black_patch = mpatches.Patch(color='k', label='Classifier Scoring')

    ax4.legend(handles=[red_patch, black_patch], loc='upper left')


    ax4.text(x_legend_offset, .5, f'Seizure onset \n per 12h', horizontalalignment='center',
    verticalalignment='center', transform=ax4.transAxes)
    ax4.text(.5, -.3, f'Recording session (12 hr)', horizontalalignment='center', 
    verticalalignment='center', transform=ax4.transAxes)
    ax4.title.set_text(f'{folder} \n State Transitions Per Recording Session')

    # if KA_flag==0:
    # plt.show()
    # plt.savefig(f'{path_figs}{use_type}frag_{folder}.jpg')
    # plt.show()
    # plt.close()




  # plt.show()
  ax4=axes[0]

  xmax=60
  ymax=300
  ymax2=60
  tick_marks=14

  n_animals+=1

  x1 = np.arange(0,len(filelist))

  # scored=count_state[:,5]==1     
  # unscored=count_state[:,5]==0

  scored=count_state[:,5]==1     
  unscored=count_state[:,5]==0

  scoredstyle='k'
  unscoredstyle='r'
  x_legend_offset=-.15

  # if n_animals==1:
  #   
  # ax4 = plt.subplot(111)
  # sz_ratio=cumu_sum[:,3]
  # y=count_state[:,3]
  y=sum(aggregate_sz[0:,:])
  # ax4.plot(sz_ratio)
  
  if len(scored)>0:
    # ax4.plot(np.ma.masked_where(scored, x1), np.ma.masked_where(scored, y), 'r')
    ax4.plot(x1,y,scoredstyle)
  # if len(unscored)>0:
  #   ax4.plot(np.ma.masked_where(unscored, x1), np.ma.masked_where(unscored, y),unscoredstyle)
  # ax4.set(xlim=(0,xmax))
  # ax4.set(ylim=(0,20))
  ax4.grid(True,'minor','x',markevery=tick_marks)
  # ax4.set_yscale('log')

  # ax5 = plt.subplot(515)
  # ax5.scatter(sum(count_state[:,3]), sum(count_state[:,0]))
  # ax5.set(xlim=(0,xmax))
  # # ax5.set(ylim=(0,ymax2))
  # ax5.grid(True,'major','x',markevery=tick_marks)

  # ax5.set_yscale('log') 
  red_patch = mpatches.Patch(color='red', label=score_label)

  black_patch = mpatches.Patch(color='k', label='Classifier Scoring')

  ax4.legend(handles=[red_patch, black_patch], loc='upper left')


  ax4.text(x_legend_offset, .5, f'Seizure onset \n per 12h', horizontalalignment='center',
  verticalalignment='center', transform=ax4.transAxes)
  ax4.text(.5, -.3, f'Recording session (12 hr)', horizontalalignment='center', 
  verticalalignment='center', transform=ax4.transAxes)
  ax4.title.set_text(f'Sum for all Animals - Sz Transitions Per Recording Session')

  plt.show()

  # ax1 = plt.subplot(511)  
  # wake_ratio=count_state[:,0]
  # ax1.plot(wake_ratio)
  #gridlines for day/week
  #72 hr after seizure

  # if Controls==False:
  #   figname=f'{path_figs}holes_paper_cs_frag_KA.jpg'
  # else:
  #   figname=f'{path_figs}holes_paper_cs_frag_Controls.jpg'

  # print(figname)
  # plt.savefig(figname, bbox_inches='tight')

#Regenerate Class Matrices from Tested Models

In [ ]:
# for type_folder in ['./Results/BiLSTM/Models/', './Results/LSTM/Models/','./Results/Dense/Models/','./Results/Quad_BiLSTM/Models/']:
for type_folder in ['./Results/Dense/Models/','./Results/Quad_BiLSTM/Models/']:

  for file_name in sorted(os.listdir(type_folder))[:]:
    if '.h5' in file_name:
      print(file_name)
      model_name=file_name.replace('.h5', '')
      model=load_model(f'{type_folder}{file_name}')

      if 'DTR+RMS only' in file_name:
        mask=mask_DTRMS
      elif 'FFT only' in file_name:
        mask=mask_FFT_only
      elif 'FullFeats' in file_name:
        mask=mask_FullFeats


      if 'win0' in file_name:
        window=0
      elif 'win1' in file_name:
        window=1
      elif 'win2' in file_name:
        window=2
      elif 'win3' in file_name:
        window=3

      print(mask, window)

      model.compile

      X_train_masked=X_train[:,mask]
      X_test_masked=X_test[:,mask]
      X_held_masked=X_held[:,mask]


      window_length=window # number of time points on either side of scored epoch to use
      # for input variables.  i.e. window_length 3 leads to an input 7 epochs long

      max_feats=X_train_masked.shape[1]

      X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
      X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
      X_held_seq=generate_sequences(X_held_masked, windows=window_length, x_or_y='X', max_feats=max_feats)

      y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
      y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
      y_held_seq=generate_sequences(y_held, windows=window_length, x_or_y='Y', max_feats=max_feats)

      savepath=type_folder.replace('Models/','')

      save_model_report_seq(model, model_history=None, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_held_seq=X_held_seq, y_train_seq=y_train_seq, y_held_seq=y_held_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                  model_name=model_name, 
                  model_num=None, basepath=basepath, logpath=None, csvpath=None, savepath=savepath);

      gc.collect()

runtime.unassign()

  
  